In [3]:
# data processing libraries
import numpy as np
import pandas as pd

# webscraping annd HTML parsing libraries
import requests
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('stopwords')
# very good tokenizer for english, considers sentence structure\n",
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB

# other libraries
import time
import os

In [4]:
response_1 = requests.get('https://www.lyrics.com/sub-artist/Elton-John/678')
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}

eltonjohn_html = response_1.text

In [5]:
response_1.status_code

200

In [6]:
with open ("eltonjohn_html", "w") as r:
    r.write(eltonjohn_html)

In [7]:
elton_soup = BeautifulSoup(eltonjohn_html,'html.parser')
type(elton_soup)

bs4.BeautifulSoup

In [9]:
elton_links = elton_soup.find(name='table', class_= 'tdata').find_all('tr')
type(elton_links)

bs4.element.ResultSet

In [10]:
pattern='''/sublyric/[^>]*"'''
song_links = []
for elton_link in elton_links:
    elton_link_str = str(elton_link) 
    song_link = re.findall(pattern, elton_link_str, re.DOTALL)
    song_links += song_link
print(song_links)

['/sublyric/4593/Elton+John/Billy+the+Kid"', '/sublyric/70090/Elton+John/Daniel"', '/sublyric/79065/Elton+John/Daniel"', '/sublyric/117902/Elton+John/DO+NOT+RUSH"', '/sublyric/63193/Elton+John/Don%27t+Let+The+Sun+Go+Down+On+Me"', '/sublyric/104941/Elton+John/I%27m+Still+Standing"', '/sublyric/73333/Elton+John/im+gonna+love+me+again"', '/sublyric/53687/Elton+John/Rocket+Man"', '/sublyric/138338/Elton+John/Tiny+Dancer"', '/sublyric/126665/Elton+John/Valhalla"', '/sublyric/4592/Elton+John/When+Billy+the+Kid+comes+to+Town"']


In [11]:
from contextlib import suppress

elton_lines = []

for counter, link in enumerate(song_links):
    with suppress(AttributeError):
        url = 'https://www.lyrics.com' + link
        response_1 = requests.get(url, headers=headers)
        response_html = response_1.text
        elton_soup=BeautifulSoup(
            markup=response_html,
            features='html.parser')
        song_text = elton_soup.find('pre',class_="lyric-body").text
        song_lines = song_text.split("\n")
        song_lines = [line.strip("\r") for line in song_lines]
        song_lines = [line for line in song_lines if len(line)>0]
        print(song_lines)
        elton_lines.extend(song_lines)

['When Billy the Kid Comes to Town']
['Daniel is traveling tonight on a plane', 'I can see the red tail lights heading for Spain', 'Oh and I can see Daniel waving goodbye', 'God it looks like Daniel, must be the clouds in my eyes', "They say Spain is pretty, though I've never been", "Well Daniel says it's the best place that he's ever seen", "Oh and he should know, he's been there enough", 'Lord I miss Daniel, oh I miss him so much', 'Oh oh, Daniel my brother you are older than me', "Do you still feel the pain of the scars that won't heal?", 'Your eyes have died, but you see more than I', "Daniel you're a star in the face of the sky", 'Oh oh, Daniel my brother you are older than me', "Do you still feel the pain of the scars that won't heal?", 'Your eyes have died, but you…']
['Daniel is travelling tonight on a plane', 'I can see the red tail lights heading for Spain', 'Oh and I can see Daniel waving goodbye', 'God it looks like Daniel, must be the clouds in my eyes', "They say Spain is

In [12]:
elton_df = pd.DataFrame({'song_lines':elton_lines})
elton_df.shape

(188, 1)

In [13]:
elton_df["singer"] = "elton_john"
elton_df

,song_lines,singer
0,When Billy the Kid Comes to Town,elton_john
1,Daniel is traveling tonight on a plane,elton_john
2,I can see the red tail lights heading for Spain,elton_john
3,Oh and I can see Daniel waving goodbye,elton_john
4,"God it looks like Daniel, must be the clouds i...",elton_john
...,...,...
183,Lay me down in sheets of linen,elton_john
184,You had a busy day today,elton_john
185,Come to Valhalla in your own time,elton_john
186,Come to Valhalla seek and you will find Valhalla,elton_john


In [14]:
elton_df.to_csv("eltonjohn_corpus.csv")

Bon Jovi

In [18]:
response_2 = requests.get('https://www.lyrics.com/album/3440435/The-Rock-History')
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}

bon_jovi_html = response_2.text

In [19]:
response_2.status_code

200

In [20]:
with open ("bon_jovi_html", "w") as r:
    r.write(bon_jovi_html)

In [21]:
bon_soup = BeautifulSoup(bon_jovi_html,'html.parser')
type(bon_soup)

bs4.BeautifulSoup

In [22]:
bon_links = bon_soup.find(name='table', class_= 'tdata').find_all('tr')
type(bon_links)
bon_links

[<tr><th>Disc</th><th>#</th><th>Song</th><th>Duration</th></tr>,
 <tr><td class="tal qx fsl">1</td><td class="tal qx fsl">1</td><td class="tal qx fsl"><div><strong><a href="/lyric/33576460/Lay+Your+Hands+on+Me">Lay Your Hands on Me</a></strong></div></td><td class="tal qx fsl">4:27</td><td class="tal"> </td></tr>,
 <tr><td class="tal qx fsl">1</td><td class="tal qx fsl">10</td><td class="tal qx fsl"><div><strong><a href="/lyric/33856808/Ride+Cowboy+Ride">Ride Cowboy Ride</a></strong></div></td><td class="tal qx fsl">2:50</td><td class="tal"> </td></tr>,
 <tr><td class="tal qx fsl">2</td><td class="tal qx fsl">2</td><td class="tal qx fsl"><div><strong><a href="/lyric/33856804/I%27d+Die+for+You">I'd Die for You</a></strong></div></td><td class="tal qx fsl">4:55</td><td class="tal"> </td></tr>,
 <tr><td class="tal qx fsl">2</td><td class="tal qx fsl">5</td><td class="tal qx fsl"><div><strong><a href="/lyric/33856799/Tokyo+Road">Tokyo Road</a></strong></div></td><td class="tal qx fsl">6:12

In [23]:
pattern='''/lyric/[^>]*"'''
song_links_2 = []
for bon_link in bon_links:
    bon_link_str = str(bon_link) 
    song_link_2 = re.findall(pattern, bon_link_str, re.DOTALL)
    song_links_2 += song_link_2
print(song_links_2)

['/lyric/33576460/Lay+Your+Hands+on+Me"', '/lyric/33856808/Ride+Cowboy+Ride"', '/lyric/33856804/I%27d+Die+for+You"', '/lyric/33856799/Tokyo+Road"', '/lyric/33856805/Raise+Your+Hands"', '/lyric/33856798/Let+It+Rock"', '/lyric/33576458/I%27ll+Be+There+for+You"', '/lyric/33856796/Blood+on+Blood"', '/lyric/33856794/Love+for+Sale"', '/lyric/33856793/Never+Say+Goodbye"', '/lyric/33856792/Diamond+Ring"', '/lyric/33856790/With+a+Little+Help+From+My+Friends"', '/lyric/33856785/You+Give+Love+a+Bad+Name"', '/lyric/33856788/Born+to+Be+My+Baby"', '/lyric/33856777/Livin%27+on+a+Prayer"', '/lyric/33576459/Bad+Medicine"', '/lyric/33856787/Wanted+Dead+or+Alive"', '/lyric/33856784/Always"', '/lyric/33856783/Roulette"', '/lyric/33856779/Runaway"']


In [24]:
song_links_2[1]

'/lyric/33856808/Ride+Cowboy+Ride"'

In [25]:
from contextlib import suppress


bon_lines = []
for counter, link in enumerate(song_links_2):
    with suppress(AttributeError):
        url_2 = 'https://www.lyrics.com' + link
        response_2 = requests.get(url_2, headers=headers)
        response_html_2 = response_2.text
        bon_soup=BeautifulSoup(
            markup=response_html_2,
            features='html.parser')
        song_text_2 = bon_soup.find('pre',class_="lyric-body").text
        song_lines_2 = song_text_2.split("\n")
        song_lines_2 = [line.strip("\r") for line in song_lines_2]
        song_lines_2 = [line for line in song_lines_2 if len(line)>0]
        print(song_lines_2)
        bon_lines.extend(song_lines_2)

['(Hey)', 'They say that to really free your body', "You've got to free your mind", 'So come on ', '(Hey, hey, hey)', 'Check this out (hey)', 'Lay your hands on me, lay your hands on me, lay your hands on me', 'Lay your hands on me, lay your hands on me, lay your hands on me', 'Come on', "If you're ready, I'm willing and able", 'Help me lay my cards out on the table', "You're mine and I'm yours for the taking", "Right now the rules we've made are meant for breaking", "What you get ain't always what you see", 'But satisfaction is guaranteed', 'They say what you give is always what you need, whoa whoa', 'If you want me to lay my hands on you', 'Lay your hands on me, lay your hands on me, lay your hands on me', 'All you gotta do is', 'Lay your hands on me, lay your hands on me, lay your hands on me', 'Now listen up', "I'm a fighter, I'm a poet, I'm a preacher", "I've been to school and baby I've been the teacher", 'If you show me how to get up off the ground', 'I can show you how to fly a

In [26]:
bon_df = pd.DataFrame({'song_lines':bon_lines})
bon_df

,song_lines
0,(Hey)
1,They say that to really free your body
2,You've got to free your mind
3,So come on
4,"(Hey, hey, hey)"
...,...
856,What was going on your head
857,"Oh, she's a little runaway"
858,Daddy's girl learned fast
859,All those things he couldn't say


In [27]:
bon_df["singer"] = "bon_jovi"
bon_df

,song_lines,singer
0,(Hey),bon_jovi
1,They say that to really free your body,bon_jovi
2,You've got to free your mind,bon_jovi
3,So come on,bon_jovi
4,"(Hey, hey, hey)",bon_jovi
...,...,...
856,What was going on your head,bon_jovi
857,"Oh, she's a little runaway",bon_jovi
858,Daddy's girl learned fast,bon_jovi
859,All those things he couldn't say,bon_jovi


In [28]:
bon_df.to_csv("bon_jovi_corpus.csv")

In [29]:
elton_bon = pd.concat([elton_df, bon_df])
elton_bon.rename(columns={"song_lines": "lyrics", "singer": "singers"}, inplace=True)
elton_bon.head()

,lyrics,singers
0,When Billy the Kid Comes to Town,elton_john
1,Daniel is traveling tonight on a plane,elton_john
2,I can see the red tail lights heading for Spain,elton_john
3,Oh and I can see Daniel waving goodbye,elton_john
4,"God it looks like Daniel, must be the clouds i...",elton_john


In [30]:
elton_bon.shape

(1049, 2)

In [31]:
corpus_all = elton_bon.copy()

corpus = []
LABELS = []

# Create list of song lines and labels
for artist in corpus_all["singers"].unique():

    # For current artist, get all song lines
    song_lines = corpus_all[corpus_all["singers"] == artist]["lyrics"]
    
    # Add each line to a list (corpus)
    for line in song_lines:
        corpus.append(line)

    # For each lyric, add the artist to another list
    for i in range(len(song_lines)):
        LABELS.append(artist)

In [32]:
corpus

['When Billy the Kid Comes to Town',
 'Daniel is traveling tonight on a plane',
 'I can see the red tail lights heading for Spain',
 'Oh and I can see Daniel waving goodbye',
 'God it looks like Daniel, must be the clouds in my eyes',
 "They say Spain is pretty, though I've never been",
 "Well Daniel says it's the best place that he's ever seen",
 "Oh and he should know, he's been there enough",
 'Lord I miss Daniel, oh I miss him so much',
 'Oh oh, Daniel my brother you are older than me',
 "Do you still feel the pain of the scars that won't heal?",
 'Your eyes have died, but you see more than I',
 "Daniel you're a star in the face of the sky",
 'Oh oh, Daniel my brother you are older than me',
 "Do you still feel the pain of the scars that won't heal?",
 'Your eyes have died, but you…',
 'Daniel is travelling tonight on a plane',
 'I can see the red tail lights heading for Spain',
 'Oh and I can see Daniel waving goodbye',
 'God it looks like Daniel, must be the clouds in my eyes',
 

In [33]:
# let's lower case everything
CORPUS = [s.lower() for s in corpus]

In [34]:
CLEAN_CORPUS = []

tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()


for doc in CORPUS:
    tokens = tokenizer.tokenize(text=doc)
    clean_doc = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    CLEAN_CORPUS.append(clean_doc)

In [36]:
CLEAN_CORPUS

['when billy the kid come to town',
 'daniel is traveling tonight on a plane',
 'i can see the red tail light heading for spain',
 'oh and i can see daniel waving goodbye',
 'god it look like daniel , must be the cloud in my eye',
 "they say spain is pretty , though i 've never been",
 "well daniel say it 's the best place that he 's ever seen",
 "oh and he should know , he 's been there enough",
 'lord i miss daniel , oh i miss him so much',
 'oh oh , daniel my brother you are older than me',
 "do you still feel the pain of the scar that wo n't heal ?",
 'your eye have died , but you see more than i',
 "daniel you 're a star in the face of the sky",
 'oh oh , daniel my brother you are older than me',
 "do you still feel the pain of the scar that wo n't heal ?",
 'your eye have died , but you…',
 'daniel is travelling tonight on a plane',
 'i can see the red tail light heading for spain',
 'oh and i can see daniel waving goodbye',
 'god it look like daniel , must be the cloud in my eye

In [37]:
STOPWORDS = stopwords.words('english')

print(STOPWORDS)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [38]:
# instanciation
vectorizer = TfidfVectorizer(stop_words=STOPWORDS)

# fit bag of words model on our corpus
vectors = vectorizer.fit_transform(CLEAN_CORPUS)


# for us to see the vectorized labeled data
pd.DataFrame(vectors.todense(), columns=vectorizer.get_feature_names_out(), index=LABELS) # check result of vectorization

,360,able,act,activity,addicted,addiction,affection,ago,ai,aim,...,wrapped,wreck,x2,yeah,year,yes,yesterday,yet,york,young
elton_john,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elton_john,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elton_john,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elton_john,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
elton_john,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bon_jovi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bon_jovi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bon_jovi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bon_jovi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
vectors.todense()[0]

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.58129211, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [40]:
model = MultinomialNB()

# fit the model with the vectorized data
model.fit(vectors, LABELS)  # model.fit

MultinomialNB()

In [41]:
LABELS

['elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_john',
 'elton_jo

In [42]:
# multinomialNB usually tends to overfit

model.score(vectors, LABELS)

0.9361296472831268

In [43]:
# TEST DATA OR NEW Data

new_lyrics = ["You had a busy day today", "another brick in the wall"]

In [44]:
new_vectors = vectorizer.transform(new_lyrics)

In [45]:
new_vectors.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
model.predict(new_vectors)

array(['elton_john', 'bon_jovi'], dtype='<U10')

In [47]:
model.predict_proba(new_vectors)

array([[0.27177428, 0.72822572],
       [0.83417628, 0.16582372]])